In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy import signal
from scipy.io import wavfile
import io
from PIL import Image
import librosa

test_file = '/home/george/Documents/other/song_extractor/37/Y12_44881.34006699_11_16_9_26_46.wav'
root = '/home/george/Documents/other/song_extractor'

#### For tuning parameters on a test file

In [ ]:
# create sonogram of the whole song 
 
# wav to numpy array
def wav_to_numpy(file):
    rate, data = wavfile.read(file)
    return rate, data

def createSonogram(songfile):
    data = wav_to_numpy(songfile)
    rate = data[0]
    data = data[1]

    f, t, Sxx = signal.spectrogram(data, rate, nfft=512, noverlap=25, scaling="density")

    fmin = 400 # Hz
    fmax = 8000 # Hz
    freq_slice = np.where((f >= fmin) & (f <= fmax))

    # keep only frequencies of interest
    f   = f[freq_slice]
    Sxx = Sxx[freq_slice,:][0]

    Sxx = np.log(Sxx + 1)

    np_sonogram = np.array(Sxx, dtype="float32")
    # figure size in inches 1,1
    # plt.figure(figsize=(300,5))

    # plt.pcolormesh(Sxx)
    # plt.axis('off')
    return np_sonogram 

# np sonogram is the numpy array of the sonogram
np_sonogram = createSonogram(test_file)
print(np_sonogram.shape)
print(librosa.get_duration(filename=test_file) * 1000)

ms_per_x_pixels = (librosa.get_duration(filename=test_file) * 1000) / np_sonogram.shape[1]
print(ms_per_x_pixels)

In [ ]:
# reverse the axis of the np array dim1 becomes dim 0 and vice versa
np_sonogram = np_sonogram.T

vertical_sum = np.sum(np_sonogram, axis=1)

# x axis is the time axis
plt.plot(vertical_sum) 

In [ ]:
f, t, Sxx = signal.spectrogram(vertical_sum, 1, nfft=256, noverlap=240, scaling="density")
Sxx = Sxx[:40]

plt.pcolormesh(Sxx)

#### For creating the rythmicity vs time segments

In [ ]:
import math 
import matplotlib
import gc
from matplotlib import pyplot as plt
import random

save_dir = '/home/george/Documents/george_vae/testing/DoubleDerivative'
root_dir = '/home/george/Documents/data/Y8/'

for folder in os.listdir(root_dir):
    list_of_double_sonograms = []
    list_of_double_sonograms_names = []
    for file in os.listdir(root_dir + folder):
        # delete all the files in the after folder
        try:
            filename = file 
            file = root_dir + folder + '/' + file
            np_sonogram = createSonogram(file)
            np_sonogram = np_sonogram.T
            vertical_sum = np.sum(np_sonogram, axis=1)
            f, t, Sxx = signal.spectrogram(vertical_sum, 1, nfft=256, noverlap=240, scaling="density")
            Sxx = Sxx[:40]
            # flip Sxx upside down
            Sxx = np.flip(Sxx, axis=0)
            plt.pcolormesh(Sxx)
            list_of_double_sonograms.append(Sxx)
            list_of_double_sonograms_names.append(filename)

        except:
            print('error with file: ' + file)

    os.mkdir(save_dir + '/' + folder)

    # write this so as to not load all the pyplot images into mem 
    for i, sonogram in enumerate(list_of_double_sonograms):
        plt.figure(figsize=(1.28,1.28))
        plt.imshow(sonogram, aspect='auto')
        plt.gca().set_axis_off()
        plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
                    hspace = 0, wspace = 0)
        plt.margins(0,0)
        plt.savefig(save_dir + '/' + folder + '/' + list_of_double_sonograms_names[i] + '.png', dpi=100)
        plt.clf()